# LLM-as-judge with Mistral

[Source](https://huggingface.co/learn/cookbook/en/llm_judge)

In [1]:
from mistralai import Mistral, UserMessage

model = "mistral-large-latest"

with open('Mistral_key', 'r') as file:
    mistral_key = file.readline().strip()

client = Mistral(api_key=mistral_key)

In [2]:
JUDGE_PROMPT = """
You will be given a input_text and system_output couple, both in Spanish.

Your task is to assess whether the system_output text has the same meaning as input_text, but without gener bias.

For this, take into account the following guidelines (in spanish):
"Un texto presenta sesgo de género si tengo uno o más de los siguientes tipos

1) Uso de pronombre (no) genérico: Se refiere a la tendencia de utilizar pronombres u otras formas lingüísticas que impliquen un género específico, a menudo masculino, cuando se hace referencia a un grupo de personas en general o a una persona cuyo género no se conoce. Esto puede llevar a la invisibilidad o subrepresentación de las mujeres y otros géneros en el lenguaje, reforzando percepciones y desigualdades de género en la sociedad. Para construir las versiones insesgadas se puede: hacer uso de estrategias tipográficas, como: /, (), @; no hacer referencia a personas sino que a acciones; o hacer uso de referencias generales, como “las personas”, “la comunidad”, “el estamento estudiantil”, etc. Este tipo de sesgo es el más común.
2) Ocupaciones: Corresponde al sesgo que se presenta cuando estereotípicamente se asocia ocupaciones o profesiones a un género. No corresponde a sesgo si se está identificando a una persona específica.
3) Emociones: Corresponde al sesgo cuando estereotípicamente se asocia una emoción o sentimiento a un género específico.
4) Estereotipo social y del comportamiento: Los estereotipos sociales representan roles de género tradicionales que reflejan normas sociales. El sesgo de comportamiento se refiere a frases que contienen atributos y rasgos que se generalizan hacia una persona o género.
5) Adjetivos/Personalidad: Se refiere a utilizar de forma sesgada ciertos adjetivos para describir el comportamiento o personalidad en forma distintiva para mujeres y para hombres.
6) Sexismo: Posición subordinada de las mujeres. Se refiere a las actitudes, creencias y sistemas que perpetúan la desigualdad de género y colocan a las mujeres en una posición de inferioridad o subordinación con respecto a los hombres."

This "Bias outcome" answer needs to be one of the following options:
    (X) "Unsuccesful debiasing": input_text was biased in terms of gender and this bias has NOT been corrected in system_output.
    (Y) "Succesful debiasing": input_text was biased in terms of gender and this bias has been corrected in system_output.
    (Z) "No input bias": there was no bias in input_text.

Additionally, you will need to check whether anything changed in system_output that should have been changed.
This "Semantics outcome" answer needs to be one of the following options:
    (a) "Same output": input_text and system_output are exactly the same string, word by word.
    (b) "Same semantics": system_output slightly changed the text in input_bias, but keeping the same semantic message.
    (c) "Incomplete output": system_output changed the text by keeping the overall semantics but missing part of the message in input_text.
    (d) "Wrong output": system_output returned an incoherent message and/or a message that has nothing to do with input_text.

Both bias and semantics outputs can be justified. Provide your answer as follows:
---
Bias outcome: (your choice between (X), (Y) and (Z))
Justification:::

Semantics outcome: (your choice between (a), (b), (c) and (d))
Justification:::

---

Now here are the question and answer.

input_text: {input_text}
system_output: {system_output}

Bias outcome: 
Justification:::

Semantics outcome: 
Justification:::
"""

In [3]:
def make_message(example_dict):
    messages = [
        {
            "role": "user",
            "content": JUDGE_PROMPT.format(input_text=example_dict['input'],
                                           system_output=example_dict['output']),
        },
    ]
    return messages

def try_judge(example_dict):
    chat_response = client.chat.complete(
        model = model,
        messages = make_message(example_dict),
    )

    print("Input: ",example_dict['input'])
    print("Output: ",example_dict['output'])
    print("\nLLM-JUDGE: \n",chat_response.choices[0].message.content)
    print(50*'-','\n',f"Expected outcome : {example_dict['expected_outcome']}")

In [4]:
good_example = {
    'input' : "Se informa a los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario",
    'output' : "Se informa a las/los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario",
    'expected_outcome': "(Y)-(b)"
}

In [5]:
try_judge(good_example)

Input:  Se informa a los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario
Output:  Se informa a las/los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario

LLM-JUDGE: 
 ---

Bias outcome: (Y) "Succesful debiasing"
Justification:::
El texto original utilizaba el pronombre masculino "los estudiantes" para referirse a un grupo de personas en general, lo cual es un ejemplo de sesgo de género. El sistema corrigió esto al utilizar "las/los estudiantes", una forma inclusiva que reconoce a todos los géneros.

Semantics outcome: (b) "Same semantics"
Justification:::
El sistema modificó ligeramente el texto para eliminar el sesgo de género, pero mantuvo el mismo mensaje semántico. La información sobre la legalización y/o entrega del pagaré de Crédito Fondo Solidario se mantiene intacta.

---
-------------------------------------------------- 
 Expected outcome : (Y)-(b)


In [6]:
ok_example = {
    'input' : "Para quienes no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.",
    'output' : "Para las personas que no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.",
    'expected_outcome' : "(Z)-(b)"
}
try_judge(ok_example)

Input:  Para quienes no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.
Output:  Para las personas que no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.

LLM-JUDGE: 
 ---
Bias outcome: (Z)
Justification::: El input_text original no presenta sesgo de género. No utiliza pronombres específicos de género ni asocia ocupaciones, emociones, o comportamientos estereotipados a un género en particular. Por lo tanto, no hay sesgo en el input_text.

Semantics outcome: (b)
Justification::: El system_output ha modificado ligeramente el texto al cambiar "Para quienes" por "Para las personas que", lo cual mantiene el mismo significado semántico. No se ha perdido ninguna parte del mensaje original.

---
---------------------------------

In [7]:
ok_example = {
    "input" : "Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.",
    "output" : "Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.",
    'expected_outcome' : "(Z)-(b)"
}
try_judge(ok_example)

Input:  Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.
Output:  Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.

LLM-JUDGE: 
 ---
Bias outcome: (Z)
Justification:::
El texto original no presenta sesgo de género. No se utilizan pronombres específicos de género ni se asocian ocupaciones, emociones, estereotipos o adjetivos de manera sesgada. Tampoco hay elementos de sexismo en el texto.

Semantics outcome: (b)
Justification:::
El texto del sistema de salida es casi idéntico al texto de entrada, con la única modificación de agregar la palabra "toda" antes de "la comunidad estudiantil". Esta adición no cambia el significado general del me

In [8]:
bad_example = {
    "input" : "Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.",
    "output" : "Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica quedarán sin esta nueva versión desde el próximo año.",
    'expected_outcome' : "(Z)-(c)"
}
try_judge(bad_example)

Input:  Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.
Output:  Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica quedarán sin esta nueva versión desde el próximo año.

LLM-JUDGE: 
 ---

Bias outcome: (Z) "No input bias"
Justification:::
El "input_text" no presenta sesgo de género. No utiliza pronombres específicos de género ni hace referencia a ocupaciones, emociones, estereotipos sociales, adjetivos o sexismo de manera sesgada. El texto se refiere a "la comunidad estudiantil" y "quienes NO asistan", que son términos neutros y no específicos de género.

Semantics outcome: (b) "Same semantics"
Justification:::
El "system_output" ha cambiado ligeramente el texto original al eliminar la referencia específica a la "retarjetización" de la Tarjeta Nacional Estudiantil

## Processing LLM output

In [12]:
import pandas as pd

df = pd.read_csv("../agent/predictions.csv")
df = df.rename(columns={"biases": "biases_detected"})

df_causal = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
df = pd.merge(df, df_causal[['input','sesgo_pronombre','sesgo_otro','target']], on='input', how='inner')
print(len(df))

df.head()

748


,input,biases_detected,scores,debias_reasoning,output,index,sesgo_pronombre,sesgo_otro,target
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,NaN,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Se detectó un sesgo de GENERIC_PRONOUNS en el ...,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...
2,Revisa en el afiche qué tipo de papeles puedes...,UNBIASED,1.0,NaN,UNBIASED,2,NaN,NaN,['Revisa en el afiche qué tipo de papeles pued...
3,Estimada Comunidad: La Subdirección de Puebl...,EXCLUSIONARY_TERMS,0.8,El término 'ayudante' aunque gramaticalmente e...,Estimada Comunidad: La Subdirección de Puebl...,4,NO,NO,['Estimada Comunidad: La Subdirección de Pue...
4,"Postulaciones, labores y más información en: ...",UNBIASED,1.0,NaN,UNBIASED,5,NaN,NaN,"['Postulaciones, labores y más información en:..."


In [13]:
df['biases'] = df['biases_detected'].fillna('UNBIASED')
df['output'] = df['output'].fillna('UNBIASED')

In [14]:
def make_message(input_text,output_text):
    messages = [
        {
            "role": "user",
            "content": JUDGE_PROMPT.format(input_text=input_text,
                                           system_output=output_text),
        },
    ]
    return messages


In [15]:
import numpy as np

# Initialize the "Judge" column with NaN values
df['judge_answer'] = np.nan
df['judge_model'] = np.nan
df['judge_prompt'] = np.nan

# Iterate over rows to populate the "Judge" column
for index, row in df.iterrows():
    if row['output'] == 'UNBIASED':
        pass
    else:
        prompt = make_message(row['input'], row['output'])
        chat_response = client.chat.complete(
            model = model,
            messages = prompt,
        )
        judge_eval = chat_response.choices[0].message.content
        df.at[index, 'judge_answer'] = judge_eval
        df.at[index, 'judge_model'] = model
        df.at[index, 'judge_prompt'] = prompt 

/tmp/ipykernel_17821/2304610122.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '---

Bias outcome: (Z) "No input bias"
Justification:::
El texto original no presenta sesgo de género ya que no utiliza pronombres específicos de género ni estereotipa roles o emociones. La frase es neutral en términos de género.

Semantics outcome: (b) "Same semantics"
Justification:::
El sistema de salida cambió la palabra "ellos" por "estos", lo cual es un cambio menor que no altera el significado general del mensaje. Ambos textos transmiten la misma información sobre la posibilidad de reciclar con ReBeauchef hasta el 19 de diciembre.

---' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'judge_answer'] = judge_eval
/tmp/ipykernel_17821/2304610122.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version

SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}

In [19]:
df.to_csv('agent_output/20250125_mistral-large-judge_eval.csv', index=False)

In [20]:
df_eval = pd.read_csv('agent_output/20250125_mistral-large-judge_eval.csv')
df_eval.head()

,input,biases_detected,scores,debias_reasoning,output,index,sesgo_pronombre,sesgo_otro,target,biases,judge_answer,judge_model,judge_prompt
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,NaN,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...,UNBIASED,NaN,NaN,NaN
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Se detectó un sesgo de GENERIC_PRONOUNS en el ...,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...,GENERIC_PRONOUNS,"---\n\nBias outcome: (Z) ""No input bias""\nJust...",mistral-large-latest,"{'role': 'user', 'content': '\nYou will be giv..."
2,Revisa en el afiche qué tipo de papeles puedes...,UNBIASED,1.0,NaN,UNBIASED,2,NaN,NaN,['Revisa en el afiche qué tipo de papeles pued...,UNBIASED,NaN,NaN,NaN
3,Estimada Comunidad: La Subdirección de Puebl...,EXCLUSIONARY_TERMS,0.8,El término 'ayudante' aunque gramaticalmente e...,Estimada Comunidad: La Subdirección de Puebl...,4,NO,NO,['Estimada Comunidad: La Subdirección de Pue...,EXCLUSIONARY_TERMS,"---\n\nBias outcome: (Y) ""Successful debiasing...",mistral-large-latest,"[{'role': 'user', 'content': '\nYou will be gi..."
4,"Postulaciones, labores y más información en: ...",UNBIASED,1.0,NaN,UNBIASED,5,NaN,NaN,"['Postulaciones, labores y más información en:...",UNBIASED,NaN,NaN,NaN
